In [1]:
# Import necessary libraries
import requests
import json

# Define the FastAPI endpoints
PREDICT_ENDPOINT = "http://127.0.0.1:8000/predict"
STORE_TWEET_ENDPOINT = "http://127.0.0.1:8000/store_tweet"

In [5]:
# Function to predict if a tweet is hate speech
def predict_tweet(text, user):
    response = requests.post(PREDICT_ENDPOINT, json={"text": text, "user": user})
    if response.status_code == 200:
        return response.json()
    else:
        print("Prediction failed:", response.json())
        return None


In [6]:
# Function to store a tweet if it's safe to post
def store_tweet(tweet_id, user_id, retweet_id=None):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id
    }
    response = requests.post(STORE_TWEET_ENDPOINT, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        print("Storing tweet failed:", response.json())
        return None

In [11]:
# Example tweet text and user
tweet_text = "This is an example tweet for testing purposes."
user_id = "example_user"

In [17]:
#Predict if the tweet is hate speech
prediction_result = predict_tweet(tweet_text, user_id)

if prediction_result:
    print("Prediction Result:", prediction_result)
    
    # Check if the tweet is safe to post (logreg_result should be 0)
    if prediction_result["logreg_result"] == 0:
        tweet_id = prediction_result["tweet_id"]
        
        # Step 3: Store the tweet in the posted tweets database
        store_result = store_tweet(tweet_id, user_id)
        if store_result:
            print("Tweet successfully posted:", store_result)
    else:
        print("Tweet is considered hate speech and will not be posted.")
else:
    print("Prediction failed.")

Prediction Result: {'tweet_id': '181512ed', 'tweet': 'This is an example tweet for testing purposes.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.01582927467824479, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None}
Tweet successfully posted: {'tweet_id': '181512ed', 'retweet_id': None, 'text': 'This is an example tweet for testing purposes.', 'retweets': 0, 'id': 1, 'user_id': 'example_user', 'likes': 0, 'safety_status': None}
